SPDX-License-Identifier: Apache-2.0
Copyright (c) 2023, Rahul Unnikrishnan Nair <rahul.unnikrishnan.nair@intel.com>


---

**Text to SQL Generation: Fine-Tuning LLMs with QLoRA on Intel**

👋 Hello and welcome! In this Jupyter Notebook, we will walkthrough the process of fine-tuning a large language model (LLM) to improve its capabilities in generating SQL queries from natural language input. The notebook is suitable for AI engineers and practitioners looking to tune LLMs for specialized tasks such as Text-to-SQL conversions.

**What you will learn with this Notebook**

- 🛠️ Fine-tune a Language Model with either a pre-existing dataset or a custom dataset tailored to your needs on Intel Hw.
- 💡 Gain insights into the fine-tuning process, including how to manipulate various training parameters to optimize your model's performance.
- 📊 Test different configurations and observe the results in real-time.

**Hardware Compatibility**

- 🖥️ Designed for 4th Generation Intel® Xeon® Scalable Processors (CPU) and Intel® Data Center GPU Max Series 1100 (XPU).

In [ ]:
!echo "List of Intel GPUs available on the system:"
!xpu-smi  discovery 2> /dev/null
!echo "Intel Xeon CPU used by this notebook:"
!lscpu | grep "Model name"

---

**Fine-Tuning with QLoRA: Balancing Memory Efficiency and Adaptability**

We leverage the QLoRA methodology for fine-tuning, enabling the loading and refinement of LLMs within the constraints of available GPU memory. Quantized Low Rank Adaptation or QLoRA achieves this by applying a clever combination of weight quantization and adapter-based finetuning.

**How Does QLoRA Work?**

- QLoRA reduces memory footprint via weight quantization. It compresses the pre-trained model weights significantly.
- During fine-tuning, it focuses on optimizing adapter parameters—low-rank matrices added to the network, tailored for the specific task.
- This selective training is computationally efficient, targeting a smaller set of trainable parameters.


**What is the Big Picture?**

- Think reparameterization: We inject LoRA weights, training only these, not the entire layer, for fine-tuning.
- This technique is key for task-specific model adaptation.
- Imagine a hub-and-spoke model for deployment: The hub is the foundational model, and the spokes are task-specific LoRA adapters.

Below, on the left, is an overview of the reparameterization implemented with LoRA (with Quantization). This involves a set of low-rank matrices—think of these as an essential subset of larger weight matrices—trained specifically for the task. On the right, there's a high-level view of a hub-and-spoke model for LLM deployment, where the hub represents the foundational model, and the spokes are the LoRA adapters.

<div align="center">
    <img src="https://github.com/rahulunair/sql_llm/assets/786476/c30d7fb4-2051-428c-9c55-fc4130cb11bc" alt="lora_adapters_reparameterization" width="75%">
</div>

## Initialization

Let's first install and import all the necessary packages required for the fine-tuning process.


In [ ]:
import sys
import site
from pathlib import Path

!echo "Installation in progress, please wait..."
!{sys.executable} -m pip cache purge > /dev/null
!{sys.executable} -m pip install --pre bigdl-llm[xpu]==2.4.0b20231116 --no-warn-script-location -f https://developer.intel.com/ipex-whl-stable-xpu > /dev/null
!{sys.executable} -m pip install peft==0.5.0 --no-deps > /dev/null
!{sys.executable} -m pip install accelerate==0.23.0 --no-deps --no-warn-script-location > /dev/null
!{sys.executable} -m pip install  transformers==4.34.0 --no-warn-script-location > /dev/null
!{sys.executable} -m pip install datasets==2.15 --no-warn-script-location > /dev/null
!{sys.executable} -m pip install fsspec==2023.9.2 > /dev/null
!echo "Installation completed."

def get_python_version():
    return "python" + ".".join(map(str, sys.version_info[:2]))

def set_local_bin_path():
    local_bin = str(Path.home() / ".local" / "bin") 
    local_site_packages = str(
        Path.home() / ".local" / "lib" / get_python_version() / "site-packages"
    )
    sys.path.append(local_bin)
    sys.path.insert(0, site.getusersitepackages())
    sys.path.insert(0, sys.path.pop(sys.path.index(local_site_packages)))

set_local_bin_path()

In [ ]:
import logging
import os
import sys
from math import ceil
from typing import Optional, Tuple
import warnings

warnings.filterwarnings(
    "ignore", category=UserWarning, module="intel_extension_for_pytorch"
)
warnings.filterwarnings(
    "ignore", category=UserWarning, module="torchvision.io.image", lineno=13
)
warnings.filterwarnings("ignore", message="You are using the default legacy behaviour")
warnings.filterwarnings("ignore", category=UserWarning, message=".*Parameter.*")
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="This implementation of AdamW is deprecated",
)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["NUMEXPR_MAX_THREADS"] = "28"
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("bigdl").setLevel(logging.ERROR)


import torch
import intel_extension_for_pytorch as ipex
from datasets import load_dataset
from datasets import Dataset
from bigdl.llm.transformers import AutoModelForCausalLM
from bigdl.llm.transformers.qlora import (
    get_peft_model,
    prepare_model_for_kbit_training as prepare_model,
)
from peft import LoraConfig
from bigdl.llm.transformers.qlora import PeftModel
import transformers
from transformers import (
    DataCollatorForSeq2Seq,
    LlamaTokenizer,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

transformers.logging.set_verbosity_error()

---

**Note on Model Storage Management**

A set of LLM foundation models are supported out-of-the-box as stated below `BASE_MDOELS` dictionary. However, if you're interested in experimenting with additional models, consider the following guidelines:

- **Storage Quota:** Be mindful of your free storage quota and space requirements for additional models.
- **PEFT Library Support:** For models supported by `peft`, refer to the [PEFT repository](https://github.com/huggingface/peft/blob/main/src/peft/utils/other.py#L434) for predefined LoRA target modules.
- **Custom Models:** For non-`peft` models, manually configure LoRA target modules in `LoraConfig`. Example for llama models: `["q_proj", "k_proj", "v_proj", "o_proj"]`.
- **Disk Space Management:** Check disk space with the provided Python function. Delete cache to free space, but this requires re-downloading models later.
- **Reset Model Cache Path:** Update `MODEL_CACHE_PATH = "~/"` in the **Model Configuration** cell.

---

**Python Function to Check Disk Space**

```python
# Function to check available disk space in the Hugging Face cache directory
import os
import shutil

def check_disk_space(path="~/.cache/huggingface/"):
    abs_path = os.path.expanduser(path)
    total, used, free = shutil.disk_usage(abs_path)
    print(f"Total: {total // (2**30)} GiB")
    print(f"Used: {used // (2**30)} GiB")
    print(f"Free: {free // (2**30)} GiB")

# Example usage
check_disk_space()
```


---

**Tailoring Your Model Configuration**

Dive into the customization core of LLM fine-tuning, equipped with a diverse range of base models to suit unique goals.

- **Model Choices in `BASE_MODELS`**: 
  - From the `open_llama_3b_v2` to the broader `Llama-2-13b-hf`.
  - Specialized options like `CodeLlama-7b-hf`.
  - Experiment to find the best fit for your objectives.

- **Dataset**:
  - Using `b-mc2/sql-create-context` from Huggingface datasets, a set of 78,577 examples (natural language queries, SQL statements).
  - Ideal for text-to-SQL models. Dataset details [here](https://huggingface.co/datasets/b-mc2/sql-create-context).

- **Your Model Options**: Within the `BASE_MODELS`, you’ll find options ranging from the nimble
  `open_llama_7b_v2` to the more expansive `Llama-2-13b-hf`, and specialized variants like `CodeLlama-7b-hf`.
  Feel free to switch between these models to discover which one aligns best with your objectives.

- **LoRA Parameters - Your Knobs to Turn**:
  - `r` (Rank): This is a key factor in how finely your model can adapt. A higher rank can grasp more
    complex nuances, while a lower rank ensures a leaner memory footprint.
  - `lora_alpha` (Scaling Factor): Adjusts LoRA adapters' impact.
    the integrity of the pre-trained weights.
  - `target_modules`: You decide which parts of the transformer model to enhance with LoRA adapters,
    directly impacting how your model interprets and generates language.
  - `lora_dropout`: Controls overfitting; experiment for optimal generalization.
  - `bias`: Modify to observe learning dynamic changes.

This notebook is set to start with `CodeLlama-7b-hf` as the default model, as our task is to generate code. To use models like Llama 2, you will have to accept the usage policy as stipulated [here](https://ai.meta.com/llama/use-policy/)


In [ ]:
BASE_MODELS = {
    "0": "NousResearch/Nous-Hermes-Llama-2-7b",  # https://huggingface.co/NousResearch/Nous-Hermes-llama-2-7b
    "1": "NousResearch/Llama-2-7b-chat-hf",  # https://huggingface.co/NousResearch/Llama-2-7b-chat-hf
    "2": "NousResearch/Llama-2-13b-hf",  # https://huggingface.co/NousResearch/Llama-2-13b-hf
    "3": "NousResearch/CodeLlama-7b-hf",  # https://huggingface.co/NousResearch/CodeLlama-7b-hf
    "4": "Phind/Phind-CodeLlama-34B-v2",  # https://huggingface.co/Phind/Phind-CodeLlama-34B-v2
    "5": "openlm-research/open_llama_3b_v2",  # https://huggingface.co/openlm-research/open_llama_3b_v2
    "6": "openlm-research/open_llama_13b",  # https://huggingface.co/openlm-research/open_llama_13b
    "7": "HuggingFaceH4/zephyr-7b-beta", # https://huggingface.co/HuggingFaceH4/zephyr-7b-beta
}
BASE_MODEL = BASE_MODELS["3"]
DATA_PATH = "b-mc2/sql-create-context"
MODEL_PATH = "./final_model"
ADAPTER_PATH = "./lora_adapters"
DEVICE = torch.device("xpu" if torch.xpu.is_available() else "cpu")
LORA_CONFIG = LoraConfig(
    r=16,  # rank
    lora_alpha=32,  # scaling factor
    target_modules=["q_proj", "k_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
MODEL_CACHE_PATH = "/home/common/data/Big_Data/GenAI/llm_models"

print("=" * 80)
print(f"Using Device: {DEVICE}")
print(f"Final model will be saved to: {MODEL_PATH}")
print(f"LoRA adapters will be saved to: {ADAPTER_PATH}")
print(f"Finetuning Model: {BASE_MODEL}")
print(f"Using dataset from: {DATA_PATH}")
print(f"Model cache: {MODEL_CACHE_PATH}")
print("=" * 80)

---

**Prompt Engineering for Text-to-SQL Conversion**

In the realm of fine-tuning language models for specialized tasks, the design of the prompt is pivotal. The function `generate_prompt_sql` encapsulates the input question, the relevant database context, and the expected output in a structured and concise manner.


In [ ]:
def generate_prompt_sql(input_question, context, output=""):
    """
    Generates a prompt for fine-tuning the LLM model for text-to-SQL tasks.

    Parameters:
        input_question (str): The input text or question to be converted to SQL.
        context (str): The schema or context in which the SQL query operates.
        output (str, optional): The expected SQL query as the output.

    Returns:
        str: A formatted string serving as the prompt for the fine-tuning task.
    """
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
{input_question}

### Context:
{context}

### Response:
{output}"""

---

**Model Loading and Configuration**

Initializing the `FineTuner`, we load the base model using `base_model_id`. Key to this setup is the `load_in_low_bit` option, using [BigDL library](https://bigdl.readthedocs.io/en/latest/) to load the model in a 4-bit format ("nf4"). This approach significantly cuts down on memory. Additionally, we configure the LoRA adapters for mixed-precision training with `torch.float16`.


In [ ]:
def setup_model_and_tokenizer(base_model_id: str):
    """Downloads / Loads the pre-trained model in NF4 datatype and tokenizer based on the given base model ID for training."""
    local_model_id = base_model_id.replace("/", "--")
    local_model_path = os.path.join(MODEL_CACHE_PATH, local_model_id)
    print(f"local model path is: {local_model_path}")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_low_bit="nf4",
            optimize_model=False,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )
    except OSError as e:
        print(e)
        sys.exit()
        logging.info(
            f"Model not found locally. Downloading {base_model_id} to cache..."
        )
        model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            load_in_low_bit="nf4",
            optimize_model=False,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )

    try:
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(local_model_path)
        else:
            tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except OSError:
        logging.info(
            f"Tokenizer not found locally. Downloading tokenizer for {base_model_id} to cache..."
        )
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
        else:
            tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"
    return model, tokenizer

---

**FineTuner**

The `FineTuner` class encapsulates the entire process of fine-tuning llms for tasks such as text-to-SQL conversion.


**Tokenization Strategy**

The tokenization process is tailored to the type of model being fine-tuned. For instance, if we are working with a Llama model, we utilize a `LlamaTokenizer` to ensure compatibility with the model's expected input format. For other models, a generic `AutoTokenizer` is used. We configure the tokenizer to pad from the left side (`padding_side="left"`) and set the pad token ID to 0.

**Data Tokenization and Preparation**

The `tokenize_data` method is where the fine-tuner ingests raw text data and converts it into a format suitable for training the model. This method handles the addition of end-of-sequence tokens, truncation to a specified `cutoff_len`, and conditioning on the input for training.

**Dataset Handling**

`prepare_data` manages the splitting of data into training and validation sets, applying the `tokenize_data` transformation to each entry. This ensures that our datasets are ready for input into the model, with all necessary tokenization applied.

**Training Process**

Finally, the `train_model` method orchestrates the training process, setting up the `Trainer` with the correct datasets, training arguments, and data collator. The fine-tuning process is encapsulated within the `finetune` method, which strings together all the previous steps into a coherent pipeline, from model setup to training execution.

**Using QLoRA for Efficient Fine-Tuning**
1. Load a pretrained model (e.g., LLaMA) in low precision with `load_in_low_bit="nf4"` for 4-bit quantized weights.
2. Prepare the quantized model with `prepare_model(model)`, handling weight quantization.
3. Add LoRA adapters via `get_peft_model(model, config)` for setting adapter parameters.
4. Fine-tune with `Trainer`, focusing gradients on adapters while keeping base model weights fixed.

**Code Implementation**
- Model loading with BigDL's `AutoModelForCausalLM`, initializing in 4-bit using `load_in_low_bit="nf4"`.
- `prepare_model()` quantizes the model weights.
- `get_peft_model()` adds LoRA adapters.
- Trainer handles fine-tuning, optimizing only adapter weights.


So in summary, we leverage QLoRA in BigDL to load the base LLM in low precision, inject adapters with `peft`, and efficiently finetune by optimizing just the adapters end-to-end while keeping the base model fixed. This unlocks huge memory savings, allowing us to adapt giant models.

In [ ]:
class FineTuner:
    """A class to handle the fine-tuning of LLM models."""

    def __init__(self, base_model_id: str, model_path: str, device: torch.device):
        """
        Initialize the FineTuner with base model, model path, and device.

        Parameters:
            base_model_id (str): Id of pre-trained model to use for fine-tuning.
            model_path (str): Path to save the fine-tuned model.
            device (torch.device): Device to run the model on.
        """
        self.base_model_id = base_model_id
        self.model_path = model_path
        self.device = device
        self.model, self.tokenizer = setup_model_and_tokenizer(base_model_id)


    def tokenize_data(
        self, data_points, add_eos_token=True, train_on_inputs=False, cutoff_len=512
    ) -> dict:
        """
        Tokenizes dataset of SQL related data points consisting of questions, context, and answers.

        Parameters:
            data_points (dict): A batch from the dataset containing 'question', 'context', and 'answer'.
            add_eos_token (bool): Whether to add an EOS token at the end of each tokenized sequence.
            cutoff_len (int): The maximum length for each tokenized sequence.

        Returns:
            dict: A dictionary containing tokenized 'input_ids', 'attention_mask', and 'labels'.
        """
        try:
            question = data_points["question"]
            context = data_points["context"]
            answer = data_points["answer"]
            if train_on_inputs:
                user_prompt = generate_prompt_sql(question, context)
                tokenized_user_prompt = self.tokenizer(
                    user_prompt,
                    truncation=True,
                    max_length=cutoff_len,
                    padding=False,
                    return_tensors=None,
                )
                user_prompt_len = len(tokenized_user_prompt["input_ids"])
                if add_eos_token:
                    user_prompt_len -= 1

            combined_text = generate_prompt_sql(question, context, answer)
            tokenized = self.tokenizer(
                combined_text,
                truncation=True,
                max_length=cutoff_len,
                padding=False,
                return_tensors=None,
            )
            if (
                tokenized["input_ids"][-1] != self.tokenizer.eos_token_id
                and add_eos_token
                and len(tokenized["input_ids"]) < cutoff_len
            ):
                tokenized["input_ids"].append(self.tokenizer.eos_token_id)
                tokenized["attention_mask"].append(1)
            tokenized["labels"] = tokenized["input_ids"].copy()
            if train_on_inputs:
                tokenized["labels"] = [-100] * user_prompt_len + tokenized["labels"][
                    user_prompt_len:
                ]

            return tokenized
        except Exception as e:
            logging.error(
                f"Error in batch tokenization: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def prepare_data(self, data, val_set_size=100) -> Dataset:
        """Prepare training and validation datasets."""
        try:
            train_val_split = data["train"].train_test_split(
                test_size=val_set_size, shuffle=True, seed=42
            )
            train_data = train_val_split["train"].shuffle().map(self.tokenize_data)
            val_data = train_val_split["test"].shuffle().map(self.tokenize_data)
            return train_data, val_data
        except Exception as e:
            logging.error(
                f"Error in preparing data: {e}, Line: {e.__traceback__.tb_lineno}"
            )
            raise e

    def train_model(self, train_data, val_data, training_args):
        """
        Fine-tune the model with the given training and validation data.

        Parameters:
            train_data (Dataset): Training data.
            val_data (Optional[Dataset]): Validation data.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            self.model = self.model.to(self.device)
            self.model = prepare_model(self.model)
            self.model = get_peft_model(self.model, LORA_CONFIG)
            trainer = Trainer(
                model=self.model,
                train_dataset=train_data,
                eval_dataset=val_data,
                args=training_args,
                data_collator=DataCollatorForSeq2Seq(
                    self.tokenizer,
                    pad_to_multiple_of=8,
                    return_tensors="pt",
                    padding=True,
                ),
            )
            self.model.config.use_cache = False
            trainer.train()
            self.model.save_pretrained(self.model_path)
        except Exception as e:
            logging.error(f"Error in model training: {e}")

    def finetune(self, data_path, training_args):
        """
        Execute the fine-tuning pipeline.

        Parameters:
            data_path (str): Path to the data for fine-tuning.
            training_args (TrainingArguments): Training configuration.
        """
        try:
            data = load_dataset(data_path)
            train_data, val_data = self.prepare_data(data)
            self.train_model(train_data, val_data, training_args)
        except KeyboardInterrupt:
            print("Interrupt received, saving model...")
            self.model.save_pretrained(f"{self.model_path}_interrupted")
            print(f"Model saved to {self.model_path}_interrupted")
        except Exception as e:
            logging.error(f"Error in fintuning: {e}")

---
**Fine-Tuning the Model**

The `lets_finetune` function orchestrates the fine-tuning process, offering a customizable interface for training. It enables specification of device, model, batch size, warm-up steps, learning rate, and maximum training steps.


**Some of the key Training Parameters:**
- `per_device_batch_size`: Number of batches on each XPU.
- `gradient_accumulation_steps`: Enables larger effective batch sizes.
- `warmup_steps`: Stabilizes training dynamics at the start.
- `save_steps`: Determines checkpoint frequency.
- `max_steps`: Limits training iterations, start with a high number like 1000 or 2000 (default here is `200`).
- `learning_rate`: Balances convergence speed and training stability.
- `max_grad_norm`: Clips gradients to avoid excessively large values.

**Monitoring and Interruption**
- Monitor training/validation loss to identify optimal stopping point.
- Interrupt training in Jupyter via `Kernel -> Interrupt Kernel` if performance is satisfactory before `max_steps`.
- Latest checkpoint is saved in `./final_model_interrupted`; last saved adapter checkpoint in `./lora_adapters`.

This setup allows for efficient and flexible model fine-tuning, adaptable to varying project needs and computational constraints.

---


In [ ]:
ENABLE_WANDB = False

def lets_finetune(
    device=DEVICE,
    model=BASE_MODEL,
    per_device_batch_size=4,
    warmup_steps=20,
    learning_rate=2e-5,
    max_steps=200,
    gradient_accum_steps=4,
):
    try:
        # Training parameters
        save_steps = 20
        eval_steps = 20
        max_grad_norm = 0.3
        save_total_limit = 3
        logging_steps = 20

        print("\n" + "\033[1;34m" + "=" * 60 + "\033[0m")
        print("\033[1;34mTraining Parameters:\033[0m")
        param_format = "\033[1;34m{:<25} {}\033[0m"
        print(param_format.format("Foundation model:", BASE_MODEL))
        print(param_format.format("Model save path:", MODEL_PATH))
        print(param_format.format("Device used:", DEVICE))
        if DEVICE.type.startswith("xpu"):
            print(param_format.format("Intel GPU:", torch.xpu.get_device_name()))
        print(param_format.format("Batch size per device:", per_device_batch_size))
        print(param_format.format("Gradient accum. steps:", gradient_accum_steps))
        print(param_format.format("Warmup steps:", warmup_steps))
        print(param_format.format("Save steps:", save_steps))
        print(param_format.format("Evaluation steps:", eval_steps))
        print(param_format.format("Max steps:", max_steps))
        print(param_format.format("Learning rate:", learning_rate))
        print(param_format.format("Max gradient norm:", max_grad_norm))
        print(param_format.format("Save total limit:", save_total_limit))
        print(param_format.format("Logging steps:", logging_steps))
        print("\033[1;34m" + "=" * 60 + "\033[0m\n")

        # Initialize the finetuner with the model and device information
        finetuner = FineTuner(
            base_model_id=model, model_path=MODEL_PATH, device=device
        )

        training_args = TrainingArguments(
            per_device_train_batch_size=per_device_batch_size,
            gradient_accumulation_steps=gradient_accum_steps,
            warmup_steps=warmup_steps,
            save_steps=save_steps,
            save_strategy="steps",
            eval_steps=eval_steps,
            evaluation_strategy="steps",
            max_steps=max_steps,
            learning_rate=learning_rate,
            #max_grad_norm=max_grad_norm,
            bf16=True,
            #lr_scheduler_type="cosine",
            load_best_model_at_end=True,
            ddp_find_unused_parameters=False,
            group_by_length=True,
            save_total_limit=save_total_limit,
            logging_steps=logging_steps,
            optim="adamw_hf",
            output_dir="./lora_adapters",
            logging_dir="./logs",
            report_to="wandb" if ENABLE_WANDB else [],
        )

        # Start fine-tuning
        finetuner.finetune(DATA_PATH, training_args)
    except Exception as e:
        logging.error(f"Error occurred: {e}")

We can optionally use Weights & Biases to track our training metrics, uncomment the below cell to enable `wandb`. You will need to pass in your API key when prompted. You can ofcourse skip this step if you'd like to.


In [ ]:
#import wandb

#os.environ["WANDB_PROJECT"] = f"text-to-sql-finetune-model-name_{BASE_MODEL.replace('/', '_')}"
#wandb.login()
#ENABLE_WANDB = True


---

**Let's Finetune!**

Now it's time to actually fine-tune the model. The `lets_finetune` function below takes care of this. It initializes a FineTuner object with the configurations you've set or left as default.

In [ ]:
lets_finetune()


**Testing our Fine-Tuned LLM**

Congratulations on successfully fine-tuning your Language Model for Text-to-SQL tasks! It's now time to put the model to the test.

___

**TextToSQLGenerator: Generating SQL Queries from Text Prompts**

**Important Note**: Remember to re-import necessary packages and re-define `BASE_MODELS` by rerunning relevant cells if the Jupyter kernel is restarted.

**Overview of `TextToSQLGenerator`**
- Designed for generating SQL queries from natural language prompts.
- Allows model selection at initialization.

**Initialization and Configuration:**
- Set `use_adapter` to `True` for using the fine-tuned LoRA model; defaults to the base model otherwise.
- Automatic tokenizer selection based on the model ID, with special handling for 'llama' models.
- Optimized loading for CPU / XPUs (`low_cpu_mem_usage`, `load_in_4bit`).
- For LoRA models, loads fine-tuned checkpoints for inference.

**Generating SQL Queries:**

The `generate` method is where the actual translation occurs. Given a text prompt, the method encodes the prompt using the tokenizer, ensuring that it fits within the model's maximum length constraints. It then performs inference to generate the SQL query.

The method parameters like `temperature` and `repetition_penalty` which we can tweak to control the creativity and quality of the generated queries!

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
INFERENCE_DEVICE = torch.device("cpu")  

def generate_prompt_sql(input_question, context, output=""):
    """
    Generates a prompt for fine-tuning the LLM model for text-to-SQL tasks.

    Parameters:
        input_question (str): The input text or question to be converted to SQL.
        context (str): The schema or context in which the SQL query operates.
        output (str, optional): The expected SQL query as the output.

    Returns:
        str: A formatted string serving as the prompt for the fine-tuning task.
    """
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
{input_question}

### Context:
{context}

### Response:
{output}"""


def setup_model_and_tokenizer(base_model_id: str):
    """Downloads / Load the pre-trained model in 4bit and tokenizer based on the given base model ID for inference."""
    local_model_id = base_model_id.replace("/", "--")
    local_model_path = os.path.join(MODEL_CACHE_PATH, local_model_id)
    print(f"local model path is: {local_model_path}")

    try:
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_4bit=True,
            optimize_model=True,
            use_cache=True,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )
    except OSError:
        logging.info(
            f"Model not found locally. Downloading {base_model_id} to cache..."
        )
        model = AutoModelForCausalLM.from_pretrained(
            local_model_path,
            load_in_4bit=True,
            optimize_model=True,
            use_cache=True,
            torch_dtype=torch.float16,
            modules_to_not_convert=["lm_head"],
        )

    try:
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(local_model_path)
        else:
            tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except OSError:
        logging.info(
            f"Tokenizer not found locally. Downloading tokenizer for {base_model_id} to cache..."
        )
        if "llama" in base_model_id.lower():
            tokenizer = LlamaTokenizer.from_pretrained(base_model_id)
        else:
            tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"
    return model, tokenizer

class TextToSQLGenerator:
    """Handles SQL query generation for a given text prompt."""

    def __init__(
        self, base_model_id=BASE_MODEL, use_adapter=False, lora_checkpoint=None
    ):
        """
        Initialize the InferenceModel class.
        Parameters:
            use_adapter (bool, optional): Whether to use LoRA model. Defaults to False.
        """
        try:
            self.model, self.tokenizer = setup_model_and_tokenizer(base_model_id)
            if use_adapter:
                self.model = PeftModel.from_pretrained(self.model, lora_checkpoint)
        except Exception as e:
            logging.error(f"Exception occurred during model initialization: {e}")
            raise

        self.model.to(INFERENCE_DEVICE)
        self.max_length = 512


    def generate(self, prompt, **kwargs):
        """Generates an SQL query based on the given prompt.
        Parameters:
            prompt (str): The SQL prompt.
        Returns:
            str: The generated SQL query.
        """
        try:
            encoded_prompt = self.tokenizer(
                prompt,
                truncation=True,
                max_length=self.max_length,
                padding=False,
                return_tensors="pt",
            ).input_ids.to(INFERENCE_DEVICE)
            with torch.no_grad():
                with torch.xpu.amp.autocast():
                    outputs = self.model.generate(
                        input_ids=encoded_prompt,
                        do_sample=True,
                        max_length=self.max_length,
                        temperature=0.3,
                        repetition_penalty=1.2,
                    )
            generated = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return generated
        except Exception as e:
            logging.error(f"Exception occurred during query generation: {e}")
            raise

---
**Generate SQL from Natural Language!** 🚀 

**With `TextToSQLGenerator`:**
- Compare base model 🆚 LoRA model.
- Instantiate with different `use_adapter` settings for side-by-side comparison.

**Things to try out:**

1. **Select a Natural Language Question** 🗣️: Use a prompt or sample data (see samples dict below) for SQL translation.
2. **Base Model SQL Generation** 🏗️: Generate SQL from the prompt using the base model.
3. **Fine-Tuned Model SQL Generation** ✨: Generate SQL with the fine-tuned model; note improvements.
4. **Compare Outputs** 🔍: Evaluate both SQL queries for accuracy to compare both models.
5. **Iterate and Refine** 🔁: Adjust training parameters or dataset and finetune again if required.
6. **Integrate with 🗂️ LlamaIndex 🦙**: Use frameworks like [LlamaIndex](https://github.com/run-llama/llama_index) to integrated your finetuned model for querying a database using natural language.


In [ ]:
import json

# let's use some fake sample data
samples = """
[
  {
    "question": "What is the capacity of the stadium where the team 'Mountain Eagles' plays?",
    "context": "CREATE TABLE stadium_info (team_name VARCHAR, stadium_name VARCHAR, capacity INT)"
  },
  {
    "question": "How many goals did player John Smith score last season?",
    "context": "CREATE TABLE player_stats (player_name VARCHAR, goals_scored INT, season VARCHAR)"
  },
  {
    "question": "What are the operating hours for the Central Library on weekends?",
    "context": "CREATE TABLE library_hours (library_name VARCHAR, day_of_week VARCHAR, open_time TIME, close_time TIME)"
  }
]
"""


def run_inference(sample_data, model):
  if INFERENCE_DEVICE.type.startswith("xpu"):
      torch.xpu.empty_cache()
  for row in sample_data:
      try:
          prompt = generate_prompt_sql(row["question"], context=row["context"])
          output = model.generate(prompt)
          print(f"\nmodel response: {output}\n")
      except Exception as e:
          logging.error(f"Exception occurred during sample processing: {e}")


# other checkpoints are saved to `./lora_adapters`, if the kernel is interrupted the latest model adapter is saved to `./final_model_interrupted/`
LORA_CHECKPOINT = "./final_model_interrupted/"
model = TextToSQLGenerator(
    use_adapter=True,
    lora_checkpoint=LORA_CHECKPOINT,
)  # set use_adapter=False to use the base model
sample_data = json.loads(samples)
run_inference(sample_data, model=model)

---
**Conclusion** 👏

We've successfully navigated the process of selecting and fine-tuning a foundational LLM model on Intel GPUs, showcasing its SQL generation capabilities. I hope that I have been able to highlight the potential of customizing language models for specific tasks and on how to efficiently finetune LLMs on Intel XPUs. As a suggestion for your continued journey, consider experimenting with different models, adjusting inference settings, and exploring various LoRA configurations to refine your results. Keep exploring!

---



**Disclaimer for Using Large Language Models**

Please be aware that while Large Language Models are powerful tools for text generation, they may sometimes produce results that are unexpected, biased, or inconsistent with the given prompt. It's advisable to carefully review the generated text and consider the context and application in which you are using these models.

For detailed information on each model's capabilities, licensing, and attribution, please refer to the respective model cards:

1. **Open LLaMA 3B v2**
   - Model Card: [openlm-research/open_llama_3b_v2](https://huggingface.co/openlm-research/open_llama_3b_v2)

2. **Open LLaMA 13B**
   - Model Card: [openlm-research/open_llama_13b](https://huggingface.co/openlm-research/open_llama_13b)

3. **Nous-Hermes LLaMA 2-7B**
   - Model Card: [NousResearch/Nous-Hermes-llama-2-7b](https://huggingface.co/NousResearch/Nous-Hermes-llama-2-7b)

4. **LLaMA 2-7B Chat HF**
   - Model Card: [NousResearch/Llama-2-7b-chat-hf](https://huggingface.co/NousResearch/Llama-2-7b-chat-hf)

5. **LLaMA 2-13B HF**
   - Model Card: [NousResearch/Llama-2-13b-hf](https://huggingface.co/NousResearch/Llama-2-13b-hf)

6. **CodeLlama 7B HF**
   - Model Card: [NousResearch/CodeLlama-7b-hf](https://huggingface.co/NousResearch/CodeLlama-7b-hf)

7. **Phind-CodeLlama 34B v2**
   - Model Card: [Phind/Phind-CodeLlama-34B-v2](https://huggingface.co/Phind/Phind-CodeLlama-34B-v2)

8. **Zephyr-7b-beta**
   - Model Card:  [HuggingFaceH4/zephyr-7b-beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta)


Usage of these models must also adhere to the licensing agreements and be in accordance with ethical guidelines and best practices for AI. If you have any concerns or encounter issues with the models, please refer to the respective model cards and documentation provided in the links above.
To the extent that any public or non-Intel datasets or models are referenced by or accessed using these materials those datasets or models are provided by the third party indicated as the content source. Intel does not create the content and does not warrant its accuracy or quality. By accessing the public content, or using materials trained on or with such content, you agree to the terms associated with that content and that your use complies with the applicable license.

 
Intel expressly disclaims the accuracy, adequacy, or completeness of any such public content, and is not liable for any errors, omissions, or defects in the content, or for any reliance on the content. Intel is not liable for any liability or damages relating to your use of public content.

Intel’s provision of these resources does not expand or otherwise alter Intel’s applicable published warranties or warranty disclaimers for Intel products or solutions, and no additional obligations, indemnifications, or liabilities arise from Intel providing such resources. Intel reserves the right, without notice, to make corrections, enhancements, improvements, and other changes to its materials.

---
